In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import numpy as np 
import scipy
import pandas as pd
from matplotlib import pyplot as plt 
import seaborn as sns
import os # imports for loading data
from tqdm import tqdm

In [ ]:
import os
import warnings

warnings.filterwarnings("ignore")

os.chdir("../../..")

In [ ]:
!pip install pytorch_forecasting

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
import torch

from pytorch_forecasting import Baseline, DeepAR, TimeSeriesDataSet
from pytorch_forecasting.data import NaNLabelEncoder
from pytorch_forecasting.data.examples import generate_ar_data
from pytorch_forecasting.metrics import SMAPE

In [ ]:
# Importing the signal in
df = pd.read_csv(os.path.abspath('/content/turn_over.csv'), index_col=False) 
print(f"Successfully read in {df.size} values.")
print("First 5 values:")
df.head(5)

In [ ]:
list_Month = [i.split(" ")[0][0:2] for i in  list(df['CALENDAR_DATE'])]
list_Date = [i.split(" ")[0][6:8] for i in  list(df['CALENDAR_DATE'])]
list_Year = [i.split(" ")[0][3:5] for i in  list(df['CALENDAR_DATE'])]

df['Month'] = list_Month
df['Date'] = list_Date
df['Year'] = list_Year

In [ ]:
df['ds'] = pd.DatetimeIndex(df['Year']+'-'+df['Month']+'-'+df['Date'])

In [ ]:
df= df.rename(columns=str.lower)

In [ ]:
df = df.drop_duplicates(inplace=False,keep='first')

In [ ]:
df.reset_index(inplace=True,drop=True)

In [ ]:
df = df.groupby(['ds','cust_id','jurisdiction'])['turnover'].sum()
df = pd.DataFrame(df)
df = df.rename_axis(['ds', 'cust_id', 'jurisdiction']).reset_index()

In [ ]:
df['ds'].agg(['min', 'max'])
#334 days`

In [ ]:
df.cust_id.nunique()

In [ ]:
juris = list(set(df['jurisdiction']))
juris

In [ ]:
df1 = df[df['jurisdiction']=='DK'].reset_index(drop=True)
df2 = df[df['jurisdiction']=='BE'].reset_index(drop=True)
df3 = df[df['jurisdiction']=='MT'].reset_index(drop=True)
df4 = df[df['jurisdiction']=='IE'].reset_index(drop=True)
df5 = df[df['jurisdiction']=='DE'].reset_index(drop=True)
df6 = df[df['jurisdiction']=='NT'].reset_index(drop=True)
df7 = df[df['jurisdiction']=='SE'].reset_index(drop=True)
df8 = df[df['jurisdiction']=='EE'].reset_index(drop=True)
df9 = df[df['jurisdiction']=='RO'].reset_index(drop=True)
df10 = df[df['jurisdiction']=='IT'].reset_index(drop=True)
df11 = df[df['jurisdiction']=='UK'].reset_index(drop=True)


In [ ]:
df

In [ ]:
df['turnover'].sum()

In [ ]:
df1.head()

In [ ]:
df1.shape

In [ ]:
df1.turnover.sum()

In [ ]:
df1.cust_id.nunique()

In [ ]:
594*334

In [ ]:
def addzeros(dataframe):
    juris = dataframe['jurisdiction'][0]
    my_dates = pd.date_range('2021-07-01', '2022-05-30', freq='D')
    idx= pd.MultiIndex.from_product([my_dates, dataframe.cust_id.unique()],names=['ds', 'cust_id'])
    dataframe = dataframe.set_index(['ds', 'cust_id']).reindex(idx).fillna(0).reset_index()
    dataframe['jurisdiction'] = juris
    return (dataframe)

In [ ]:
df1 = addzeros(df1)
df2 = addzeros(df2)
df3 = addzeros(df3)
df4 = addzeros(df4)
df5 = addzeros(df5)
df6 = addzeros(df6)
df7 = addzeros(df7)
df8 = addzeros(df8)
df9 = addzeros(df9)
df10 = addzeros(df10)
df11 = addzeros(df11)

In [ ]:
dataframe = pd.concat([df1, df2,df3,df4,df5,df6,df7,df8,df9,df10,df11], ignore_index=True)

In [ ]:
dataframe

In [ ]:
data = dataframe

In [ ]:
data = data.astype(dict(cust_id=str))

In [ ]:
data['ds'] = pd.to_datetime(data['ds'] )

In [ ]:
min_date = data['ds'].min()
data["time_idx"] = data["ds"].map(lambda current_date: (current_date - min_date).days)

In [ ]:
# create dataset and dataloaders
max_encoder_length = 20
max_prediction_length = 5

training_cutoff = data["time_idx"].max() - max_prediction_length

context_length = max_encoder_length
prediction_length = max_prediction_length

training = TimeSeriesDataSet(
    data[lambda x: x.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="turnover",
    group_ids=["cust_id"],
    # static_categoricals=[
    #     "jurisdiction"
    # ],  # as we plan to forecast correlations, it is important to use series characteristics (e.g. a series identifier)
    time_varying_unknown_reals=["turnover"],
    time_varying_known_reals=["time_idx"],
    max_encoder_length=context_length,
    max_prediction_length=prediction_length
)

In [ ]:
validation = TimeSeriesDataSet.from_dataset(training, data, min_prediction_idx=training_cutoff + 1)
batch_size = 128

In [ ]:
batch_size = 128
train_dataloader = training.to_dataloader(
    train=True, batch_size=batch_size, num_workers=3
)
val_dataloader = validation.to_dataloader(
    train=False, batch_size=batch_size, num_workers=3
)

In [ ]:
pl.seed_everything(42)
import pytorch_forecasting as ptf

trainer = pl.Trainer(gpus=0, gradient_clip_val=1e-1,limit_train_batches=200,max_epochs = 10)
net = DeepAR.from_dataset(
    training, learning_rate=1e-2, hidden_size=30, rnn_layers=2, 
)

In [ ]:
res = trainer.tuner.lr_find(
    net,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
    min_lr=1e-8,
    max_lr=1e0,
    early_stop_threshold=100,
)
print(f"suggested learning rate: {res.suggestion()}")
fig = res.plot(show=True, suggest=True)
fig.show()
net.hparams.learning_rate = res.suggestion()

In [ ]:
# early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
# trainer = pl.Trainer(
#     max_epochs=3,
#     gpus=0,
#     enable_model_summary=True,
#     gradient_clip_val=0.1,
#     callbacks=[early_stop_callback],
#     limit_train_batches=200,
#     enable_checkpointing=True,
# )


# net = DeepAR.from_dataset(
#     training,
#     learning_rate=res.suggestion(),
#     log_interval=10,
#     log_val_interval=1,
#     hidden_size=30,
#     rnn_layers=2)

trainer.fit(
    net,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

In [ ]:
best_model_path = trainer.checkpoint_callback.best_model_path
best_model = DeepAR.load_from_checkpoint(best_model_path)

In [ ]:
actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
predictions = best_model.predict(val_dataloader)
(actuals - predictions).abs().mean()

In [ ]:
((actuals - predictions)**2).mean()

In [ ]:
raw_predictions, x = net.predict(val_dataloader, mode="raw", return_x=True)

In [ ]:
raw_predictions[0][0][0]